In [1]:
import pandas as pd
import numpy as np
import requests
from lxml import html, etree
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.text, "html.parser")

###### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
post_table = soup.find("table",class_="wikitable sortable")

In [6]:
table_list = []
for rows in post_table.find_all('tr'):
    if rows.find_all('th'):
        postal_code = rows.find_all('th')[0].get_text()
        borough = rows.find_all('th')[1].get_text()
        neighborhood = rows.find_all('th')[2].get_text()[:-1]
    else:
        if rows.find_all('td')[1].a:
            postal_code = rows.find_all('td')[0].get_text()
            borough = rows.find_all('td')[1].get_text()
            neighborhood = rows.find_all('td')[2].get_text()[:-1]
        else:
            continue
    table_list.append([postal_code, borough, neighborhood])

In [7]:
df = pd.DataFrame(table_list)
df.columns = df.iloc[0]
df = df[1:]
df.head(10)

,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Rouge
10,M1B,Scarborough,Malvern


In [8]:
df = df.rename(columns={'Postcode':'PostalCode', 'Borough':'Borough', 'Neighbourhood':'Neighborhood'})

###### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [9]:
new_df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
new_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


###### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [10]:
new_df[new_df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
83,M7A,Queen's Park,Not assigned


In [11]:
for i, row in new_df.iterrows():
    if new_df['Neighborhood'][i] == 'Not assigned':
        new_df['Neighborhood'][i] = new_df['Borough'][i]

In [12]:
new_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
